## CIA READINGROOM > CREST > SCIENTIFIC ABSTRACTS 

Scraping tests for metadata + pdf retrieval (pre-OCR tasks)<br>10.17.17

In [1]:
from bs4 import BeautifulSoup
from os.path import expanduser
import os
from tqdm import tqdm
import pandas as pd
import requests
import urllib2
import time
import lxml

In [2]:
BASEURL = "https://www.cia.gov"
PAGINATE_PATH ="/library/readingroom/collection/scientific-abstracts?page="
PDF_DIR = expanduser("~") + "/cia_pdfs/"
RANGE = 1654 # pages, of 20 docs per page
TEST_RANGE = 10
SKIPPED_FILES = []

In [3]:
def retrieve_file(url, name):
    filepath = PDF_DIR + name + ".pdf"
    if not os.path.exists(PDF_DIR):
        os.makedirs(PDF_DIR)
    try:
        response = urllib2.urlopen(url)
        with open(filepath, 'w+') as f:
            f.write(response.read())
            f.close()
    except urllib2.URLError as e:
        print ('WiFi connection perhaps lost !! Trying one more time...')
        try:
            response = urllib2.urlopen(url)
            with open(filepath, 'w+') as f:
                f.write(response.read())
                f.close()
        except:
            print ('WiFi connection really lost !! Bailing out..')
            print (e)
            SKIPPED_FILES.append(name)

In [4]:
def sciab_df(download_bool):
    
    df = pd.DataFrame(columns=['id','title','classification','publication_date'])
    idx = 0
    
    for i in tqdm(xrange(TEST_RANGE)):
        
        pagination_link = BASEURL + PAGINATE_PATH + str(i)
        pagination_page = requests.get(pagination_link)
        p_soup = BeautifulSoup(pagination_page.content, 'lxml')
        
        for doc_title in p_soup.find_all("h4", class_="field-content"):
            
            a = doc_title.select_one("a")
            link = str(a.get('href'))
            TITLE = str(a.string or "")
            
            doc_page = requests.get(BASEURL + link)
            m_soup = BeautifulSoup(doc_page.content, 'lxml')
            time.sleep (.05)
        
            try:
                PUB_DATE = str(m_soup.select_one(".field-name-field-pub-date").select_one("span").get('content') or "")
            except:
                print "Invalid PUB_DATE field. Continuing."
                SKIPPED_FILES.append(name)
            try:
                ID = str(m_soup.select_one(".field-name-field-document-number").select_one(".field-item").string or "")
            except:
                print "Invalid ID field. Continuing."
                SKIPPED_FILES.append(name)
            try:
                CLASSIFICATION = str(m_soup.select_one(".field-name-field-original-classification").select_one(".field-item").string or "")
            except:
                print "Invalid CLASSIFICATION field. Continuing."
                SKIPPED_FILES.append(name)
                
            if download_bool:
                PDF = m_soup.select_one(".file").select_one("a").get('href')
                retrieve_file(PDF, ID)
            
            df.loc[idx] = [ID, TITLE, CLASSIFICATION, PUB_DATE]
            idx+=1  
            
    return df

In [5]:
sciab = sciab_df(False)

100%|██████████| 10/10 [00:43<00:00,  4.45s/it]


In [6]:
sciab[:30]

,id,title,classification,publication_date
0,CIA-RDP86-00513R001031010018-6,"""SCIENTIFIC ABSTRACT LVOVSKIY, P.G. - LYADOVA...",S,1967-12-31T01:01:01-05:00
1,CIA-RDP86-00513R000204310003-8,"""SCIENTIFIC ABSTRACT BELENKIY, M.N. - BELENKI...",S,1967-12-31T01:01:01-05:00
2,CIA-RDP86-00513R000204310001-0,"""SCIENTIFIC ABSTRACT BELENKIY, G. A. - BELENKI...",S,1967-12-31T01:01:01-05:00
3,CIA-RDP86-00513R000204310002-9,"""SCIENTIFIC ABSTRACT BELENKIY, L.I. - BELENKIY...",S,1967-12-31T01:01:01-05:00
4,CIA-RDP86-00513R000204310004-7,"""SCIENTIFIC ABSTRACT BELENKIY, N. - BELENKIY, ...",S,1967-12-31T01:01:01-05:00
5,CIA-RDP86-00513R000204310005-6,"""SCIENTIFIC ABSTRACT BELENKIY, V. I. - BELENKI...",S,1967-12-31T01:01:01-05:00
6,CIA-RDP86-00513R000204310006-5,"""SCIENTIFIC ABSTRACT BELENKIY, Y. Y. - BELENOV...",S,1967-12-31T01:01:01-05:00
7,CIA-RDP86-00513R000204310007-4,"""SCIENTIFIC ABSTRACT BELENKOV, V.P. - BELENOVS...",S,1967-12-31T01:01:01-05:00
8,CIA-RDP86-00513R000204310008-3,"""SCIENTIFIC ABSTRACT BELENSCHCHIKOV, L.D. - BE...",S,1967-12-31T01:01:01-05:00
9,CIA-RDP86-00513R000204310009-2,"""SCIENTIFIC ABSTRACT BELETSKAYA, I.P. - BELETS...",S,1967-12-31T01:01:01-05:00
